## Import the original JSON files

In [196]:
from binaryninja.architecture import Architecture
from binaryninja.platform import Platform
from binaryninja.typelibrary import TypeLibrary
from binaryninja.enums import StructureVariant, NamedTypeReferenceClass
from binaryninja.types import (Type, Tuple, PointerType, StructureType, EnumerationType, FunctionType, EnumerationMember, EnumerationBuilder, QualifiedName, FunctionParameter, PointerBuilder)
from binaryninja.log import log_to_stdout
from binaryninja.enums import LogLevel
from functools import cache
from typing import Optional, Set, List
from pathlib import Path
import json, codecs
from collections import defaultdict

log_to_stdout(LogLevel.WarningLog)
api_namespaces = {}
for file in Path("win32json/api/").glob("*.json"):
    with codecs.open(str(file), "r", "utf-8-sig") as f:
        api_namespaces[file.stem] = json.load(f)

## Create Ordinal/GUID Mapping

In [197]:
from tqdm.notebook import tqdm

mapping = {}
items = list(Path("/Users/peterlafosse/src/binaryninja/typelib/x86_64/").glob("*.dll.bntl"))
for file in tqdm(items, total=len(items), unit="typelibs", desc="building ordinal mapping file"):
    filename = file.name
    file = str(file)
    tl = TypeLibrary.load_from_file(file)
    if tl is None:
        print(file)
    assert tl is not None, f"Failed to open {file}"
    ordinals_name = tl.query_metadata("ordinals")
    ordinals = tl.query_metadata(str(ordinals_name))
    mapping[filename] = {}
    mapping[filename]["guid"] = tl.guid
    mapping[filename]["ordinals"] = ordinals_name
    mapping[filename][ordinals_name] = ordinals
with open("mappingfile.json", "w") as f:
    json.dump(mapping, f)

building ordinal mapping file:   0%|          | 0/30 [00:00<?, ?typelibs/s]

## Remap the functions by DllImport

In [327]:
from collections import defaultdict
libs = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
types = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
arch_list = ["Arm64", "X64", "X86"]
def assign(archs:List, api:str, type_name:str, type):
    global types
    global libs
    if len(archs) > 0:
        for arch in type["Architectures"]:
            types[arch][api][type_name] = type
    else:
        for arch in arch_list:
            types[arch][api][type_name] = type

def composite_type(type, prefix):
    for nested_type in type["NestedTypes"]:
        new_prefix = f"{prefix}{nested_type['Name']}"
        assign(nested_type["Architectures"], name, new_prefix, nested_type)
        composite_type(nested_type, new_prefix + "::")

for name, obj in api_namespaces.items():
    for func in obj["Functions"]:
        dll = func["DllImport"].lower()
        if len(func["Architectures"]) > 0:
            for arch in func["Architectures"]:
                libs[arch][dll][name][func["Name"]] = func
        else:
            for arch in arch_list:
                libs[arch][dll][name][func["Name"]] = func

    for type in obj["Types"]:
        assign(type["Architectures"], name, type["Name"], type)
        if type["Kind"] in ("Struct", "Union"):
            composite_type(type, f"{type['Name']}::")

# Spot check expected types
assert "APPLICATION_RECOVERY_CALLBACK" in types['X86']['System.WindowsProgramming']
assert "PROCESS_HEAP_ENTRY" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union::_Block_e__Struct" in types["X64"]["System.Memory"]
assert "PROCESS_HEAP_ENTRY::_Anonymous_e__Union::_Block_e__Struct::HANDLE" not in types["X64"]["System.Memory"]
assert "HANDLE" in types["X64"]["Foundation"]

# Verify that each type definition is reachable
def verify(api, arch, t, name):
    kind = t["Kind"]
    if kind in ("PointerTo", "Array", "LPArray"):
        verify(api, arch, t["Child"], name)
    elif kind == "ApiRef":
        try:
            types[arch][t["Api"]][t["Name"]]
        except KeyError:
            types[arch][t["Api"]][f"{name}::{t['Name']}"]
    elif kind in ("Struct", "Union"):
        for type in t["NestedTypes"]:
            verify(api, arch, type, f"{name}::{t['Name']}" if name is None else name)

for arch, apis in types.items():
    for api, type_dict in apis.items():
        for name, t in type_dict.items():
            verify(api, arch, t, name)

## Create a reverse mapping of api,type_name
So we can figure out which are the common types

In [362]:
def deps(types, t, prefix, seen, cur_api, nested=False) -> Set[Tuple[str, str]]:
    assert isinstance(t, dict)
    kind = t["Kind"]
    if kind == "Native":
        return set()
    elif t["Kind"] == "ApiRef":
        item = (t['Api'], prefix + t["Name"])
        result = set([item])
        if item not in seen:
            seen.add(item)
            if nested:
                result |= deps(types, [t['Api']][prefix + t["Name"]], prefix, seen, t['Api'])
        return result
    elif t["Kind"] == "NativeTypedef":
        return deps(types, t["Def"], prefix, seen, cur_api)
    elif t["Kind"] == "Array":
        return deps(types, t["Child"], prefix, seen, cur_api)
    elif t["Kind"] == "Enum":
        return set([(cur_api, f"{prefix}{t['Name']}")])
    elif t["Kind"] in ("Struct", "Union"):
        result = set()
        for type in t["NestedTypes"]:
            result |= deps(types, type, prefix + f"{t['Name']}::", seen, cur_api, True)
        for field in t["Fields"]:
            result |= deps(types, field["Type"], "", seen, cur_api)
        return result
    elif t["Kind"] == "FunctionPointer":
        result = set()
        for field in t["Params"]:
            result |= deps(types, field["Type"], prefix, seen, cur_api)
        result |= deps(types, t["ReturnType"], prefix, seen, cur_api)
        return result
    elif t["Kind"] == "Com":
        result = set()
        for method in t["Methods"]:
            for param in method["Params"]:
                result |= deps(types, param["Type"], prefix, seen, cur_api)
            result |= deps(types, method["ReturnType"], prefix, seen, cur_api)
        return result
    elif t["Kind"] in ("LPArray", "PointerTo"):
        return deps(types, t["Child"], prefix, seen, cur_api)
    else:
        return set()

types_needed = defaultdict(lambda: {})
for arch, func_info in libs.items():
    for lib_name, func_list in func_info.items():
        types_needed[arch][lib_name] = []
        seen = set()
        for api, func_dict in func_list.items():
            for func_name, func in func_dict.items():
                if func is None:
                    print(func_name)
                for param in func["Params"]:
                    types_needed[arch][lib_name].extend(deps(types, param["Type"], "", seen, api))
                types_needed[arch][lib_name].extend(deps(types, func["ReturnType"], "", seen, api))

needed_by = defaultdict(lambda: defaultdict(lambda: set()))
for arch in arch_list:
    for name, t in types_needed[arch].items():
        for api, type_name in t:
            needed_by[arch][(api, type_name)].add(name)

most_common_types = defaultdict(lambda: set())
dependencies = defaultdict(lambda: set())
for arch in arch_list:
    for (api, type_name), names in needed_by[arch].items():
        if len(names) > 1:
            most_common_types[arch].add((api, type_name))
            dependencies[arch] |= deps(types["X64"], types["X64"][api][type_name], "", set(), api)

for arch in arch_list:
    print(arch)
    print(f"  {sum([len(t) for t in types[arch].values()])} - Count of all types")
    print(f"  {len(most_common_types[arch])}  - Count of types which appear in more than one library")
    print(f"  {len(dependencies[arch])} - Dependencies of these common types")
    most_common_types[arch] |= dependencies[arch]
    print(f"  {len(most_common_types[arch])} - Dependencies and common types")

for arch, total in zip(arch_list, [33278, 33279, 33268]):
    cur_total = sum([len(type) for type in types[arch].values()])
    assert cur_total == total, f"Total {arch} types not equal to {total} instead {cur_total}"
print("Success")

Arm64
  33278 - Count of all types
  269  - Count of types which appear in more than one library
  364 - Dependencies of these common types
  523 - Dependencies and common types
X64
  33279 - Count of all types
  269  - Count of types which appear in more than one library
  363 - Dependencies of these common types
  522 - Dependencies and common types
X86
  33268 - Count of all types
  268  - Count of types which appear in more than one library
  362 - Dependencies of these common types
  520 - Dependencies and common types
Success


In [216]:
from binaryninja.types import QualifiedName

# def create_type(typelib: TypeLibrary, t, needed:Set[str], prefix="") -> Type:
#     assert isinstance(t, dict)
#     kind = t["Kind"]
#     if kind == "Native":
#         name = t["Name"]
#         if name == "Byte":
#             return Type.int(1, sign=False)
#         elif name == "SByte":
#             return Type.int(1)
#         elif name == "Char":
#             return Type.char()
#         elif name == "UInt16":
#             return Type.int(2, sign=False)
#         elif name == "Int16":
#             return Type.int(2)
#         elif name == "Int64":
#             return Type.int(8)
#         elif name == "UInt32":
#             return Type.int(4, sign=False)
#         elif name == "UInt64":
#             return Type.int(8, sign=False)
#         elif name == "Int32":
#             return Type.int(4)
#         elif name == "Single":
#             return Type.float(4)
#         elif name == "Double":
#             return Type.float(8)
#         elif name == "UIntPtr":
#             return Type.pointer(typelib.arch, Type.int(8, sign=False))
#         elif name == "IntPtr":
#             return Type.pointer(typelib.arch, Type.int(8, sign=True))
#         elif name == "Void":
#             return Type.void()
#         elif name == "Boolean":
#             return Type.bool()
#         elif name == "Guid":
#             #FIXME
#             return Type.void()
#         else:
#             assert False, f"Unhandled Native Type: {name}"
#     elif t["Kind"] == "ApiRef":
#         if typelib.name != "win32common" and (t['Api'], t['Name']) in most_common_types:
#             typelib.add_type_source(QualifiedName(t["Name"]), "win32common")
#             return Type.named_type_reference(NamedTypeReferenceClass.TypedefNamedTypeClass, t["Name"], f"win32common:{prefix}{t['Name']}")
#         old_type = typelib.get_named_type(t["Name"])
#         if old_type is not None:
#             return old_type

#         needed.add((t["Api"], t["Name"]))
#         return Type.named_type_reference(NamedTypeReferenceClass.TypedefNamedTypeClass, t["Name"], f"{t['Api']}:{prefix}{t['Name']}")
#     elif t["Kind"] == "NativeTypedef":
#         new_type = create_type(typelib, t["Def"], needed, prefix)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type)
#         return new_type
#     elif t["Kind"] == "Array":
#       if t["Shape"]:
#         child = create_type(typelib, t["Child"], needed, prefix)
#         assert child is not None
#         return Type.array(child, int(t["Shape"]["Size"]))
#       else:
#         child = create_type(typelib, t["Child"], needed, prefix)
#         assert child is not None
#         return Type.pointer(typelib.arch, child)
#     elif t["Kind"] == "Enum":
#         members = [EnumerationMember(str(member["Name"]), int(member["Value"])) for member in t["Values"]]
#         new_type = EnumerationType.create(members, width=8, arch=typelib.arch)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type)
#         return new_type
#     elif t["Kind"] == "Struct":
#         members = []
#         for field in t["Fields"]:
#             name = field["Name"]
#             # This is a big hack and should be done recursively somehow it might break in the future
#             for nested_type in t["NestedTypes"]:
#                 if field["Type"]["Kind"] == "ApiRef" and nested_type["Name"] == field["Type"]["Name"]:
#                     type = create_type(typelib, nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#                 if field["Type"]["Kind"] in ("PointerTo", "Array") and field["Type"]["Child"]["Kind"] == "ApiRef" and nested_type["Name"] == field["Type"]["Child"]["Name"]:
#                     type = create_type(typelib, nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#             else:
#                 if field["Type"]["Kind"] == "ApiRef":
#                     type = create_type(typelib, types[field["Type"]["Api"]][field["Type"]["Name"]], needed, prefix + f"{t['Name']}::")
#                 else:
#                     type = create_type(typelib, field["Type"], needed, prefix + f"{t['Name']}::")
#             members.append((type, name))
#         new_type = StructureType.create(members)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type) # TODO packing?
#         return new_type
#     elif t["Kind"] == "Union":
#         members = []
#         for field in t["Fields"]:
#             name = field["Name"]
#             for nested_type in t["NestedTypes"]:
#                 if field["Type"]["Kind"] == "ApiRef" and nested_type["Name"] == field["Type"]["Name"]:
#                     type = create_type(typelib, nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#                 if field["Type"]["Kind"] in ("PointerTo", "Array") and field["Type"]["Child"]["Kind"] == "ApiRef" and nested_type["Name"] == field["Type"]["Child"]["Name"]:
#                     type = create_type(typelib, nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#             else:
#                 if field["Type"]["Kind"] == "ApiRef":
#                     type = create_type(typelib, types[field["Type"]["Api"]][field["Type"]["Name"]], needed, prefix + f"{t['Name']}::")
#                 else:
#                     type = create_type(typelib, field["Type"], needed, prefix + f"{t['Name']}::")
#             members.append((type, name))
#         new_type = StructureType.create(members, False, StructureVariant.UnionStructureType)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type) # TODO packing?
#         return new_type
#     elif t["Kind"] == "FunctionPointer":
#         params = [(str(param["Name"]), create_type(typelib, param["Type"], needed, prefix)) for param in t["Params"]]
#         target = FunctionType.create(create_type(typelib, t["ReturnType"], needed, prefix), params)
#         new_type = PointerType.create(typelib.arch, target)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type)
#         return new_type
#     elif t["Kind"] == "Com":
#         members = [] # TODO: Probably need to not do this
#         for method in t["Methods"]:
#             params = []
#             for param in method["Params"]:
#                 param_type = create_type(typelib, param["Type"], needed, prefix)
#                 assert param_type is not None
#                 params.append(FunctionParameter(param_type, param["Name"]))
#             return_type = create_type(typelib, method["ReturnType"], needed, prefix)
#             assert return_type is not None
#             func = FunctionType.create(return_type, params)
#             members.append((Type.pointer(typelib.arch, func), method["Name"]))
#         new_type = StructureType.create(members)
#         assert new_type is not None
#         typelib.add_named_type(t["Name"], new_type)
#         return new_type
#     elif t["Kind"] in ("LPArray", "PointerTo"):
#         new_type = create_type(typelib, t["Child"], needed, prefix)
#         assert new_type is not None, t["Child"]
#         return Type.pointer(typelib.arch, new_type)
#     elif t["Kind"] == "ComClassID":
#         return Type.void() # TODO: What is this really?
#     elif t["Kind"] == "MissingClrType":
#         return Type.void()
#     else:
#         print(f"Found unknown type kind: {t['Kind']}")

In [ ]:
# from typing import DefaultDict, Dict
# defined_types:DefaultDict[str, DefaultDict[str, DefaultDict[str, Type]]] = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))

# def pointer_width(arch:str):
#     return 4 if arch == 'X86' else 8

# def define_type(arch, api:str, t, needed:Set[str], prefix="", seen={}) -> Type:
#     assert isinstance(t, dict), f"{t}"
#     if "Name" in t:
#         type = defined_types[arch][api][t["Name"]]
#         if type is not None:
#             return type

#     kind = t["Kind"]
#     if kind == "Native":
#         name = t["Name"]
#         type = defined_types[arch][api][name]
#         if type is not None:
#             return type

#         if name == "Byte":
#             return Type.int(1, sign=False)
#         elif name == "SByte":
#             return Type.int(1)
#         elif name == "Char":
#             return Type.char()
#         elif name == "UInt16":
#             return Type.int(2, sign=False)
#         elif name == "Int16":
#             return Type.int(2)
#         elif name == "Int64":
#             return Type.int(8)
#         elif name == "UInt32":
#             return Type.int(4, sign=False)
#         elif name == "UInt64":
#             return Type.int(8, sign=False)
#         elif name == "Int32":
#             return Type.int(4)
#         elif name == "Single":
#             return Type.float(4)
#         elif name == "Double":
#             return Type.float(8)
#         elif name == "UIntPtr":
#             return Type.pointer(None, Type.int(8, sign=False), width=8)
#         elif name == "IntPtr":
#             return Type.pointer(None, Type.int(8, sign=True), width=8)
#         elif name == "Void":
#             return Type.void()
#         elif name == "Boolean":
#             return Type.bool()
#         elif name == "Guid":
#             #FIXME
#             return Type.void()
#         else:
#             assert False, f"Unhandled Native Type: {name}"
#     elif kind == "ApiRef":
#         needed.add((t["Api"], t["Name"]))
#         return Type.named_type_reference(NamedTypeReferenceClass.TypedefNamedTypeClass, t["Name"], f"t['Api']:{prefix}{t['Name']}")
#     elif kind == "NativeTypedef":
#         new_type = define_type(arch, api, t["Def"], needed, prefix)
#         return Type.named_type_from_type_and_id(f"t['Api']:{prefix}{t['Name']}", t["Name"], new_type)
#     elif kind == "Array":
#       if t["Shape"]:
#         child = define_type(arch, api, t["Child"], needed, prefix)
#         assert child is not None
#         return Type.array(child, int(t["Shape"]["Size"]))
#       else:
#         child = define_type(arch, api, t["Child"], needed, prefix)
#         assert child is not None
#         return Type.pointer_of_width(pointer_width(arch), child)
#     elif kind == "Enum":
#         members = [EnumerationMember(str(member["Name"]), int(member["Value"])) for member in t["Values"]]
#         new_type = EnumerationType.create(members, width=8)
#         assert new_type is not None
#         return new_type
#     elif kind in ("Struct", "Union"):
#         # TODO: Deal with packing!!
#         members = []
#         for field in t["Fields"]:
#             field_type = field["Type"]
#             field_name = field["Name"]
#             # First see if the field type is a nested type
#             for nested_type in t["NestedTypes"]:
#                 # this is rather hacky and should probably be done a better way recursively
#                 # this represents all the different ways a structure can reference a nested structure/union
#                 if field_type["Kind"] == "ApiRef" and nested_type["Name"] == field_type["Name"]:
#                     type = define_type(arch, field_type["Api"], nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#                 if field_type["Kind"] in ("PointerTo", "Array") and field_type["Child"]["Kind"] == "ApiRef" and nested_type["Name"] == field_type["Child"]["Name"]:
#                     type = define_type(arch, api, nested_type, needed, prefix + f"{t['Name']}::")
#                     break
#             else:
#                 # its not a nested type
#                 if field_type["Kind"] == "ApiRef":
#                     try:
#                         item = types[arch][field_type["Api"]][field_type["Name"]]
#                     except KeyError:
#                         print(arch, field_type["Api"], field_type["Name"])
#                     if (arch, field_type["Api"], field_type["Name"]) in seen:
#                         type = seen[(arch, field_type["Api"], field_type["Name"])]
#                     else:
#                         type = define_type(arch, field_type["Api"], item, needed, prefix + f"{t['Name']}::", seen)
#                         seen[(arch, field_type["Api"], field_type["Name"])] = type
#                 else:
#                     type = define_type(arch, api, field_type, needed, prefix + f"{t['Name']}::")
#             members.append((type, field_name))
#         new_type = StructureType.create(members, t["PackingSize"] == 0, StructureVariant.UnionStructureType if kind == "Union" else StructureVariant.StructStructureType)
#         assert new_type is not None
#         return new_type
#     elif kind == "FunctionPointer":
#         params = []
#         for param in t["Params"]:
#             params.append((param["Name"], define_type(arch, api, param["Type"], needed, prefix)))

#         target = FunctionType.create(define_type(arch, api, t["ReturnType"], needed, prefix), params)
#         new_type = Type.pointer_of_width(pointer_width(arch), target)
#         assert new_type is not None
#         return new_type
#     elif kind == "Com":
#         return Type.void()
#         # TODO: Probably need to actually define these
#         # members = []
#         # for method in t["Methods"]:
#         #     params = []
#         #     for param in method["Params"]:
#         #         param_type = define_type(arch, api, param["Type"], needed, prefix)
#         #         assert param_type is not None
#         #         params.append(FunctionParameter(param_type, param["Name"]))
#         #     return_type = define_type(arch, api, method["ReturnType"], needed, prefix)
#         #     assert return_type is not None
#         #     func = FunctionType.create(return_type, params)
#         #     members.append((Type.pointer_of_width(pointer_width(arch), func), method["Name"]))
#         # new_type = StructureType.create(members)
#         # assert new_type is not None
#         # return new_type
#     elif kind in ("LPArray", "PointerTo"):
#         new_type = define_type(arch, api, t["Child"], needed, prefix)
#         assert new_type is not None, t["Child"]
#         return Type.pointer_of_width(pointer_width(arch), new_type)
#     elif kind == "ComClassID":
#         return Type.void() # TODO: What is this really?
#     elif kind == "MissingClrType":
#         return Type.void()
#     else:
#         print(f"Found unknown type kind: {t['Kind']}")


# for arch, apis in types.items():
#     print(f"Constructing {arch}")
#     for api, api_types in apis.items():
#         for name, type in api_types.items():
#             if type is None:
#                 if not name.startswith("_"):
#                     print("asdf", arch, api, name)
#                 continue
#             defined_types[arch][api][name] = define_type(arch, api, type, set(), "")

Constructing Arm64
Constructing X64
Constructing X86


In [369]:
lib_sets = {
    ("X64", "x86_64", Architecture["x86_64"], Platform["windows-x86_64"], "winX64common"),
    ("X86", "x86", Architecture["x86"], Platform["windows-x86"], "win32common"),
    ("Arm64", "aarch64", Architecture["aarch64"], Platform["windows-aarch64"], "winArm64common"),
}
from binaryninja.enums import TypeClass
from binaryninja.types import NamedTypeReferenceType

defined_types:DefaultDict[str, DefaultDict[str, DefaultDict[str, Type]]] = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
def add_to_type_library(typelib:TypeLibrary, arch_name:str, api_name:str, type_name:Optional[QualifiedName], t, seen:Set[Tuple[str, str, str]], dependent:Optional[TypeLibrary] = None, nested_types = None) -> Optional[Type]:
    kind = t["Kind"]
    if kind == "Native":
        name = t["Name"]
        type = defined_types[arch_name][api_name][name]
        if type is not None:
            return type
        if name == "Byte":
            return Type.int(1, sign=False)
        elif name == "SByte":
            return Type.int(1)
        elif name == "Char":
            return Type.char()
        elif name == "UInt16":
            return Type.int(2, sign=False)
        elif name == "Int16":
            return Type.int(2)
        elif name == "Int64":
            return Type.int(8)
        elif name == "UInt32":
            return Type.int(4, sign=False)
        elif name == "UInt64":
            return Type.int(8, sign=False)
        elif name == "Int32":
            return Type.int(4)
        elif name == "Single":
            return Type.float(4)
        elif name == "Double":
            return Type.float(8)
        elif name == "UIntPtr":
            # TODO: Ensure integer width is correct
            return Type.pointer_of_width(pointer_width(arch), Type.int(pointer_width(arch), sign=False))
        elif name == "IntPtr":
            # TODO: Ensure integer width is correct
            return Type.pointer_of_width(pointer_width(arch), Type.int(pointer_width(arch), sign=True))
        elif name == "Void":
            return Type.void()
        elif name == "Boolean":
            return Type.bool()
        elif name == "Guid":
            #FIXME
            return Type.void()
        else:
            assert False, f"Unhandled Native Type: {name}"
    elif kind == "ApiRef":
        name = t["Name"]
        api = t["Api"]

        if nested_types is not None:
            for nested_type in nested_types:
                if name == nested_type["Name"]:
                    seen.add((arch_name, api, name))
                    result = add_to_type_library(typelib, arch_name, api_name, name, nested_type, seen, dependent, nested_types)
                    assert result is not None
                    return result

        if dependent is not None and dependent.get_named_type(name) is not None:
            result = dependent.get_named_type(name)
            dep_name = dependent.name
            assert dep_name is not None
            if name == "SECURITY_DESCRIPTOR":
                typelib.add_type_source(QualifiedName("ACL"), dep_name)
            assert result is not None
            typelib.add_named_type(name, result)
            typelib.add_type_source(QualifiedName(name), dep_name)
            assert result is not None
            return result

        if (arch_name, api, name) in seen:
            # we have a recursively defined type
            return Type.named_type_reference(NamedTypeReferenceClass.TypedefNamedTypeClass, name, f"{api}:{name}")
        new_type_info = types[arch_name][api][name]
        assert new_type_info is not None, f"{arch_name, api, name} returned None"
        seen.add((arch_name, api, name))
        new_type = add_to_type_library(typelib, arch_name, api, name, new_type_info, seen, dependent, nested_types)
        assert new_type is not None
        typelib.add_named_type(name, new_type)
        return new_type
    elif kind == "NativeTypedef":
        name = t["Name"]
        if dependent is not None and dependent.get_named_type(name) is not None:
            result = dependent.get_named_type(name)
            assert result is not None
            dep_name = dependent.name
            assert dep_name is not None
            typelib.add_type_source(QualifiedName(name), dep_name)
            assert result is not None
            return result
        seen.add((arch_name, api_name, name))
        new_type = add_to_type_library(typelib, arch_name, api_name, None, t["Def"], seen, dependent, nested_types)
        result = Type.named_type_from_type_and_id(f"{api_name}:{name}", name, new_type)
        typelib.add_named_type(name, result)
        return result
    elif kind == "Array":
        child = t["Child"]
        if t["Shape"]:
            child = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
            assert child is not None
            return Type.array(child, int(t["Shape"]["Size"]))
        else:
            child = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
            assert child is not None
            return Type.pointer_of_width(pointer_width(arch), child)
    elif kind == "Enum":
        name = t["Name"]
        type = defined_types[arch_name][api_name][name]
        if type is not None:
            typelib.add_named_type(name, type)
            return type
        seen.add((arch_name, api_name, name))
        members = [EnumerationMember(str(member["Name"]), int(member["Value"])) for member in t["Values"]]
        result = EnumerationType.create(members, width=8)
        assert result is not None
        typelib.add_named_type(name, result)
        defined_types[arch_name][api_name][name] = result
        return result
    elif kind in ("Struct", "Union"):
        name = t["Name"]
        members = []
        for field in t["Fields"]:
            type = add_to_type_library(typelib, arch_name, api_name, None, field["Type"], seen, dependent, t["NestedTypes"])
            assert type is not None, f"returned none: {arch_name} {api_name}, {field['Type']}"
            members.append((type, field["Name"]))
        result = StructureType.create(members, t["PackingSize"] == 0, StructureVariant.UnionStructureType if kind == "Union" else StructureVariant.StructStructureType)
        assert result is not None
        typelib.add_named_type(name, result)
        return result
    elif kind == "FunctionPointer":
        name = t["Name"]
        params = []
        for param in t["Params"]:
            type = add_to_type_library(typelib, arch_name, api_name, None, param["Type"], seen, dependent, nested_types)
            params.append((param["Name"], type))
        return_type = add_to_type_library(typelib, arch_name, api_name, None, t["ReturnType"], seen, dependent, nested_types)
        target = FunctionType.create(return_type, params)
        result = Type.pointer_of_width(pointer_width(arch), target)
        assert result is not None
        typelib.add_named_type(name, result)
        return result
    elif kind == "Com":
        return Type.void()
        # TODO: Probably need to actually define these
        # members = []
        # for method in t["Methods"]:
        #     params = []
        #     for param in method["Params"]:
        #         param_type = define_type(arch, api, param["Type"], needed, prefix)
        #         assert param_type is not None
        #         params.append(FunctionParameter(param_type, param["Name"]))
        #     return_type = define_type(arch, api, method["ReturnType"], needed, prefix)
        #     assert return_type is not None
        #     func = FunctionType.create(return_type, params)
        #     members.append((Type.pointer_of_width(pointer_width(arch), func), method["Name"]))
        # new_type = StructureType.create(members)
        # assert new_type is not None
        # return new_type
    elif kind in ("LPArray", "PointerTo"):
        new_type = add_to_type_library(typelib, arch_name, api_name, None, t["Child"], seen, dependent, nested_types)
        assert new_type is not None, t["Child"]
        result = Type.pointer_of_width(pointer_width(arch), new_type)
        assert result is not None
        return result
    elif kind == "ComClassID":
        return Type.void() # TODO: What is this really?
    elif kind == "MissingClrType":
        return Type.void()
    else:
        print(f"Found unknown type kind: {t['Kind']}")

win_common = {}
for key, arch_name, arch, platform, bntl_name in lib_sets:
    defined_types = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None)))
    win_common[key] = TypeLibrary.new(arch, bntl_name)
    win_common[key].add_platform(platform)
    filename = f"output/{arch_name}/{bntl_name}.bntl"
    print(len(most_common_types[key]), key, arch_name, platform, bntl_name, filename)

    for api, type_name in most_common_types[key]:
        type = types[key][api][type_name]
        if type is None:
            if not type_name.startswith("_"):
                print("couldn't find: ", key, api, type_name)
            continue
        t = add_to_type_library(win_common[key], key, api, QualifiedName(type_name), type, set())
        if t is not None:
            win_common[key].add_named_type(QualifiedName(type_name), t)
    win_common[key].finalize()
    win_common[key].write_to_file(filename)

for arch, total in zip(arch_list, [661, 662, 660]):
    cur_total = len(win_common[arch].named_types)
    assert cur_total == total, f"Total for {arch} doesn't match {total} instead {cur_total}"
print("Success")


520 X86 x86 windows-x86 win32common output/x86/win32common.bntl
522 X64 x86_64 windows-x86_64 winX64common output/x86_64/winX64common.bntl
523 Arm64 aarch64 windows-aarch64 winArm64common output/aarch64/winArm64common.bntl
Success


## Build the TypeLibraries

In [370]:
from typing import Set, List

ordinal_data = {}
ordinal_data["Arm64"] = {}
ordinal_data["X86"] = {}
ordinal_data["X64"] = json.load(open("mappingfile.json"))
typelibs = {}
for key, arch_name, arch, platform, dependency_name in tqdm(lib_sets, total=len(lib_sets), desc=f"Building Type Libraries"):
    for lib_name, func_list in tqdm(libs[key].items(), total=len(libs[key]), unit="typelibs", desc=f"Building {arch_name} Type Libraries"):
        typelib = TypeLibrary.new(arch, f"{lib_name.lower()}.dll")
        typelib.add_platform(platform)
        typelib.dependency_name = dependency_name
        lookup_name = f"{lib_name.lower()}.dll.bntl"
        if lookup_name in ordinal_data[key]:
            typelib.guid = str(ordinal_data[key][lookup_name]["guid"])
            ordinal_name = ordinal_data[key][lookup_name]["ordinals"]
            if ordinal_name is not None:
                typelib.store_metadata("ordinals", ordinal_name)
                typelib.store_metadata(ordinal_name, ordinal_data[key][lookup_name][ordinal_name])
        for api_name, func_dict in func_list.items():
            for func_name, func in func_dict.items():
                params = [(param["Name"], add_to_type_library(typelib, key, api_name, None, param["Type"], set(), win_common[key])) for param in func["Params"]]
                return_type = add_to_type_library(typelib, key, api_name, None, func["ReturnType"], set(), win_common[key])
                function_type = Type.function(return_type, params)
                typelib.add_named_object(func_name, function_type)

        for name in win_common[key].named_types.keys():
            typelib.add_type_source(QualifiedName(name), win_common[key].name)

        typelib.finalize()
        typelib.write_to_file(f"output/{arch_name}/{lookup_name}")
        typelibs[f"output/{arch_name}/{lookup_name}"] = typelib

Building Type Libraries:   0%|          | 0/3 [00:00<?, ?it/s]

Building x86 Type Libraries:   0%|          | 0/337 [00:00<?, ?typelibs/s]

Building x86_64 Type Libraries:   0%|          | 0/337 [00:00<?, ?typelibs/s]

Building aarch64 Type Libraries:   0%|          | 0/337 [00:00<?, ?typelibs/s]

## TypeLibrary Checks

HRESULT (uint32_t dwFlags, uint32_t dwTimeout, uint32_t cHandles, HANDLE* pHandles, uint32_t* lpdwindex)
HRESULT (uint32_t dwFlags, uint32_t dwTimeout, uint32_t cHandles, HANDLE* pHandles, uint32_t* lpdwindex)
